In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

# 1. Training components

In [84]:
token_to_id = {
    'what' : 0,
    'is' : 1,
    'transformer' : 2,
    'magic' : 3,
    '<EOS>' : 4
}

id_to_token = dict(map(reversed, token_to_id.items()))

In [85]:
inputs = torch.tensor([[token_to_id["what"],
                        token_to_id["is"],
                        token_to_id["transformer"],
                        token_to_id["<EOS>"],
                        token_to_id["magic"]],

                       [token_to_id["transformer"],
                        token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["magic"]]])

labels = torch.tensor([[token_to_id["is"],
                        token_to_id["transformer"],
                        token_to_id["<EOS>"],
                        token_to_id["magic"],
                        token_to_id["<EOS>"]],

                       [token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["magic"],
                        token_to_id["<EOS>"]]])

In [86]:
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

# position encoding
We'll use Sinusoidal Positional Encoding

Given a position \( pos \) and dimension \( i \), the positional encoding is defined as:

$$
\text{PE}(pos, 2i) = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$

$$
\text{PE}(pos, 2i+1) = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$



In [110]:
class PositionEncoding(nn.Module):
    def __init__(self, d_model=2, max_len=6):
      super().__init__()
      position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)

      embedding_index = torch.arange(start=0, end=d_model, step=2).float()

      div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)

      pe = torch.zeros(max_len, d_model)
      pe[:, 0::2] = torch.sin(position * div_term)
      pe[:, 1::2] = torch.cos(position * div_term)

      self.register_buffer('pe', pe)

    def forward(self, word_embeddings):
      return word_embeddings + self.pe[:word_embeddings.size(0), :]


# Attention Mechanism
* Q,K,V values

In [111]:
class Attention(nn.Module):
  def __init__(self, d_model=2):
    super().__init__()

    self.W_q = nn.Linear(in_features= d_model, out_features=d_model, bias=False)
    self.W_k = nn.Linear(in_features= d_model, out_features=d_model, bias=False)
    self.W_v = nn.Linear(in_features= d_model, out_features=d_model, bias=False)

    self.row_dim=0
    self.col_dim=1

  def forward(self,encoding_for_q, encoding_for_k, encoding_for_v,mask=None):

    q = self.W_q(encoding_for_q)
    k = self.W_k(encoding_for_k)
    v = self.W_v(encoding_for_v)

    sims = torch.matmul(q,k.transpose(dim0=self.row_dim,dim1=self.col_dim))

    #normalization
    scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

    if mask is not None:
      scaled_sims = scaled_sims.masked_fill(mask == 0, -1e9)

    attention_weights = F.softmax(scaled_sims, dim=1)

    #returning attention score
    return torch.matmul(attention_weights,v)

# 2.Decoder only transformer

In [89]:
!pip install lightning

In [90]:
import lightning as l

In [115]:
class DecoderOnlyTransformer(l.LightningModule):
    def __init__(self, num_tokens=4, d_model=2, max_len=6):
        super().__init__()

        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)
        self.pe = PositionEncoding(d_model=d_model, max_len=max_len)
        self.attention = Attention(d_model=d_model)
        self.fc = nn.Linear(in_features=d_model, out_features=num_tokens)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):
        word_embeddings = self.we(token_ids)
        position_encoded = self.pe(word_embeddings)

        mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0)), device=self.device))
        mask = mask == 0

        attention_values = self.attention(position_encoded, position_encoded, position_encoded, mask=mask)
        residual_connection_values = attention_values + position_encoded
        logits = self.fc(residual_connection_values)

        return logits

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_idx):
        input_tokens, labels = batch
        output = self.forward(input_tokens[0])
        loss = self.loss(output.view(-1, output.size(-1)), labels[0].view(-1))
        return loss

# Running Before training

In [116]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

In [93]:
model_input = torch.tensor([token_to_id["what"],
                            token_to_id["is"],
                            token_to_id["transformer"],
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

max_length = 6
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 <EOS>


# 3. Traning the model

In [117]:
trainer = l.Trainer(max_epochs=30)
trainer.fit(model, train_dataloaders=dataloader)

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | we      

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=30` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


# 4. Running the model after training

In [118]:
model_input = torch.tensor([token_to_id["what"],
                            token_to_id["is"],
                            token_to_id["transformer"],
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

max_length = 6
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 magic
	 <EOS>


## Now that the model gives the correct token as prediction we'll try it for different token sequence

In [119]:
model_input = torch.tensor([
                            token_to_id["transformer"],
                            token_to_id["is"],
                            token_to_id["what"],
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

max_length = 6
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 magic
	 <EOS>


## Technical Summary of the Decoder-Only Transformer

This notebook implements a simplified decoder-only transformer model using PyTorch and Lightning. The model is designed for a small vocabulary and fixed sequence length, demonstrating the core components of a transformer architecture.

**Key Components:**

1.  **Tokenization and Embedding:**
    *   A fixed vocabulary is defined with a mapping from tokens to IDs (`token_to_id`) and vice-versa (`id_to_token`).
    *   An `nn.Embedding` layer (`self.we`) converts input token IDs into dense word embeddings of a specified dimension (`d_model`).

2.  **Positional Encoding:**
    *   A `PositionEncoding` module adds positional information to the word embeddings, as transformers do not inherently process sequential order.
    *   This implementation uses a sinusoidal positional encoding scheme, where different dimensions of the encoding correspond to sinusoids of varying frequencies. The positional encodings are pre-calculated and added to the word embeddings.

3.  **Attention Mechanism:**
    *   An `Attention` module implements the scaled dot-product attention mechanism.
    *   It calculates Query (Q), Key (K), and Value (V) vectors from the input using linear transformations (`self.q`, `self.k`, `self.v`).
    *   Attention weights are computed by taking the dot product of Q and K, scaling by the square root of the model dimension (`d_model`), and applying a softmax function.
    *   A causal mask is applied to the attention weights during training to prevent the model from attending to future tokens in the sequence, which is characteristic of decoder-only models.
    *   The output of the attention layer is a weighted sum of the Value vectors, where the weights are the attention weights.

4.  **Decoder-Only Architecture:**
    *   The `DecoderOnlyTransformer` class combines the embedding, positional encoding, and attention layers.
    *   The `forward` method takes a sequence of token IDs as input.
    *   It first obtains word embeddings and adds positional encodings.
    *   Then, it passes the position-encoded embeddings through the attention layer with a causal mask.
    *   A residual connection is applied by adding the attention output to the position-encoded input.
    *   Finally, a linear layer (`self.fc`) projects the output of the attention mechanism to the size of the vocabulary, producing logits for the next token prediction.

5.  **Training with PyTorch Lightning:**
    *   The model is wrapped in a PyTorch Lightning module, providing structure for training.
    *   The `configure_optimizers` method defines the Adam optimizer.
    *   The `training_step` method calculates the cross-entropy loss between the model's output logits and the target labels.
    *   A Lightning `Trainer` is used to handle the training loop, including optimization and loss calculation over epochs.

**Limitations of this Implementation:**

*   **Simplified Attention:** This implementation uses a single attention head and does not include multi-head attention.
*   **No Layer Normalization or Feed-Forward Networks:** Standard transformer blocks typically include layer normalization and feed-forward neural networks after the attention mechanism, which are omitted here for simplicity.
*   **Small Scale:** The model and vocabulary size are very small, suitable for demonstrating the architecture but not for practical natural language processing tasks.
*   **Basic Training Loop:** The training loop is minimal and does not include validation or advanced logging as typically found in more complete Lightning examples.